<a href="https://colab.research.google.com/github/elisasanzani/SoftwareCarpentry/blob/main/lr_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/lr_finder_2.py -O lr_finder.py

--2022-08-13 13:57:58--  https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/lr_finder_2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1474 (1.4K) [text/plain]
Saving to: ‘lr_finder.py’

lr_finder.py        100%[===================>]   1.44K  --.-KB/s    in 0s      

2022-08-13 13:57:58 (17.6 MB/s) - ‘lr_finder.py’ saved [1474/1474]



In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

!wget https://www.dropbox.com/sh/oqlkgpajx6nu8i7/AABnJFTfOheRs2O3x_NT_q72a?dl=1 -O keras_input.zip
!unzip keras_input.zip

data_folder = "."

name_map = {"points": "xy", "features": "et", "mask": "mask", "summary": "etrn"}

signal  = {key: np.load(f"{data_folder}/signal_{ name_map[key]}.npy") for key in name_map}

mnbs    = {key: np.load(f"{data_folder}/mnbs_{   name_map[key]}.npy") for key in name_map}

cosmics = {key: np.load(f"{data_folder}/cosmics_{name_map[key]}.npy") for key in name_map}

signal_weight = 330/150e3 * 100
mnbs_weight = 0.24 * 100
cosmics_weight = 4/53e3 * 100


--2022-08-13 13:57:59--  https://www.dropbox.com/sh/oqlkgpajx6nu8i7/AABnJFTfOheRs2O3x_NT_q72a?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/oqlkgpajx6nu8i7/AABnJFTfOheRs2O3x_NT_q72a [following]
--2022-08-13 13:58:00--  https://www.dropbox.com/sh/dl/oqlkgpajx6nu8i7/AABnJFTfOheRs2O3x_NT_q72a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc32d7ae8a615c48a2bff8011906.dl.dropboxusercontent.com/zip_download_get/BOXlqJRCXkwnfi4qF6Hyj56eaT3Zc9uRvRODA-dJtsjBb3IMPyaZMl3dYEhjubLPvPMll4pGch-I1WF8Dg2TUbecWY2rc3sbY494h9ScJxXhsg?dl=1# [following]
--2022-08-13 13:58:00--  https://uc32d7ae8a615c48a2bff8011906.dl.dropboxusercontent.com/zip_download_get/BOXlqJRCXkwnfi4qF6Hyj56eaT3Zc9uRvRODA-dJtsjBb3IMPyaZMl3dYEhjubLPvPMll4pGch-I1

In [3]:
S =  signal["mask"].shape[0] 
F =    mnbs["mask"].shape[0] 
C = cosmics["mask"].shape[0]

N = S + F + C

p = np.random.permutation(N) 

data = {key: np.concatenate( (signal[key], mnbs[key], cosmics[key]) )[p] for key in name_map}

label_list = [1, 0, 0]
entries_list = [S, F, C]
weight_list = [signal_weight, mnbs_weight, cosmics_weight]

print(f"- Entries per signale, mnbs e cosmici:{entries_list}")
print(f"- Pesi per signale, mnbs e cosmici:{weight_list}")

label = np.concatenate( 
    [ np.ones((e))*l for e, l in zip(entries_list,  label_list) ]
)[p]

sample_weight = np.concatenate( 
    [ np.ones((e))*w for e, w in zip(entries_list, weight_list) ]
)[p]

from sklearn.preprocessing import StandardScaler
import copy
scaler = StandardScaler()

virgin_data = copy.deepcopy(data)

for key in ["points", "features"]:
  data[key][data["mask"].reshape(N, 50)] = scaler.fit_transform(data[key][data["mask"].reshape(N, 50)])

data["summary"] = scaler.fit_transform(data["summary"])

train_data  = {key: data[key][int(N*0.0) : int(N*0.7)] for key in name_map}
test_data   = {key: data[key][int(N*0.7) : int(N*0.9)] for key in name_map}
val_data    = {key: data[key][int(N*0.9) : int(N*1.0)] for key in name_map}

train_label = label[int(N*0.0) : int(N*0.7)]
test_label  = label[int(N*0.7) : int(N*0.9)]
val_label   = label[int(N*0.9) : int(N*1.0)]

train_weight = sample_weight[int(N*0.0) : int(N*0.7)]
test_weight  = sample_weight[int(N*0.7) : int(N*0.9)]
val_weight   = sample_weight[int(N*0.9) : int(N*1.0)]


- Entries per signale, mnbs e cosmici:[41372, 1132, 41598]
- Pesi per signale, mnbs e cosmici:[0.22, 24.0, 0.007547169811320755]


In [4]:
# downloading custom particle_net_lite keras model from out github repo
!wget https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py -O tf_keras_model.py

import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net_lite_custom

input_shapes = {'points': (50, 2), 'features': (50, 2), 'mask':(50, 1)}

model = get_particle_net_lite_custom(input_shapes)

batch_size = 1024
epochs = 30



--2022-08-13 13:58:07--  https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6767 (6.6K) [text/plain]
Saving to: ‘tf_keras_model.py’

tf_keras_model.py   100%[===================>]   6.61K  --.-KB/s    in 0s      

2022-08-13 13:58:08 (32.8 MB/s) - ‘tf_keras_model.py’ saved [6767/6767]



In [5]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(),
              weighted_metrics=[keras.metrics.AUC(curve="PR")],
)
model.summary()

Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 50, 1)]      0           []                               
                                                                                                  
 tf.math.not_equal (TFOpLambda)  (None, 50, 1)       0           ['mask[0][0]']                   
                                                                                                  
 tf.cast (TFOpLambda)           (None, 50, 1)        0           ['tf.math.not_equal[0][0]']      
                                                                                                  
 tf.math.equal (TFOpLambda)     (None, 50, 1)        0           ['tf.cast[0][0]']                
                                                                                        

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

class LRFind(tf.keras.callbacks.Callback): 
    def __init__(self, min_lr, max_lr, n_rounds): 
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lrs = []
        self.losses = []
        self.lrs_history = []
        self.losses_history = []
        self.n_rounds = n_rounds
        self.best_lr = 0

    def on_train_begin(self, logs=None):
      self.model.optimizer.lr = self.min_lr

    def on_epoch_begin(self, epoch, logs=None):
        if epoch %2 == 0:
          self.weights = self.model.get_weights()
          self.lrs = []
          self.losses = []
          self.step_up = (self.max_lr / self.min_lr) ** (1 / self.n_rounds)

        else:
          self.step_up = 1
          self.model.optimizer.lr = self.best_lr
          print(f"Best LR: {self.best_lr}")

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 2 == 0:
          self.model.set_weights(self.weights)
          self.best_lr = self.lrs[np.argmin(self.losses)]*1e-1
          self.lrs_history.append(self.lrs)
          self.losses_history.append(self.losses)

    def on_train_batch_end(self, batch, logs=None):
        self.lrs.append(self.model.optimizer.lr.numpy())
        self.losses.append(logs["loss"])
        self.model.optimizer.lr = self.model.optimizer.lr * self.step_up
 
        if self.model.optimizer.lr > self.max_lr:
            self.model.stop_training = True

In [ ]:
EPOCHS = 15
BATCH_SIZE = 1024
lr_finder_steps = 400
lr_find = LRFind(1e-6, 1e1, lr_finder_steps)
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label))\
                    .repeat()\
                    .shuffle(2048)\
                    .batch(BATCH_SIZE)
model.fit(
    train_dataset,
    steps_per_epoch=lr_finder_steps,
    epochs=EPOCHS,
    callbacks=[lr_find]
)

plt.plot(lr_find.lrs, lr_find.losses)
plt.xscale('log')
plt.show()

Epoch 1/15
 86/400 [=====>........................] - ETA: 3:36 - loss: 0.6776 - auc: 0.5502